In [23]:
# default_exp getheightmap

In [21]:
#hide
from nbdev.showdoc import *

In [142]:
# export 
import cv2
import numpy as np
import imageio

def bwareaopen(image, sz):
    output = image.copy()
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    for i in range(1,nlabels-1):
        regions_size = stats[i,4]
        if regions_size < sz:
            x0 = stats[i,0]
            y0 = stats[i,1]
            x1 = stats[i,0]+stats[i,2]
            y1 = stats[i,1]+stats[i,3]
            for row in range(y0, y1):
                for col in range(x0, x1):
                    if labels[row, col] == i:
                        output[row, col] = 0

    return output

def bg_subtraction(colorIMg, bgColorImg, depthImg, bgDepthImg):
    tmp = abs(colorIMg - bgColorImg)
    tmp[tmp<0.3] == 1

    fgMaskColor = np.sum(tmp,2)
    fgMaskColor[fgMaskColor!=3] = 1
    fgMaskColor[fgMaskColor==3] = 0

    tmp = abs(depthImg-bgDepthImg)
    tmp[tmp>0.02] = 1
    tmp[tmp!=1] = 0

    bgDepthImg[bgDepthImg!=0] = 1

    fgMaskDepth = np.zeros(bgDepthImg.shape)
    fgMaskDepth[((tmp == 1) & (bgDepthImg == 1))] = 1

    fgMask = np.ones(bgDepthImg.shape)
    fgMask[((fgMaskDepth == 0) & (fgMaskColor == 0))] = 0

    return fgMaskColor, fgMask

def project_depth2camera(camIntrinsics, camPose, depthImg):
    xvalues = np.linspace(1,640,640)
    yvalues = np.linspace(1,480,480)
    pixX,pixY = np.meshgrid(xvalues, yvalues)

    camX = (pixX - camIntrinsics[0][2])*(depthImg/camIntrinsics[0][0])
    camY = (pixY - camIntrinsics[1][2])*(depthImg/camIntrinsics[1][1])
    camZ = depthImg
    camPts = np.array([camX.flatten('F').transpose(),camY.flatten('F').transpose(), camZ.flatten('F').transpose()])
    camPts = camPts.transpose()

    return camPts

def points2world(camPose, camPts):
    g = np.expand_dims(camPose[0:3,3], axis=1)
    tmp = np.repeat(g,307200, axis=1).transpose()
    tmp2 = np.dot(camPose[0:3,0:3], camPts.transpose()).transpose()
    worldPts = tmp2 + tmp

    return worldPts

def getgridmapping(worldPts, binMiddleBottom):
    voxelSize = 0.002
    gridOrigin = [binMiddleBottom[0][0]-0.3,binMiddleBottom[0][1]-0.2,binMiddleBottom[0][2]]
    gridMapping_1 = np.around((worldPts[:,0] - gridOrigin[0])/voxelSize)
    gridMapping_2 = np.around((worldPts[:,1] - gridOrigin[1])/voxelSize)
    gridMapping_3 = (worldPts[:,2] - binMiddleBottom[0][2])
    gridMapping = np.array([gridMapping_1, gridMapping_2, gridMapping_3]).transpose()

    return gridMapping

def getheightmapColor(gridMapping, colorIMg):
    heightMap = np.zeros((200,300))
    heightMapColor = np.zeros((200*300,3))

    Tmp0 = np.zeros(gridMapping[:,0].shape)
    Tmp0[gridMapping[:,0]>0] = 1

    Tmp1 = np.zeros(gridMapping[:,0].shape)
    Tmp1[gridMapping[:,0]<= 300] = 1

    Tmp2 = np.zeros(gridMapping[:,1].shape)
    Tmp2[gridMapping[:,1] > 0] = 1

    Tmp3 = np.zeros(gridMapping[:,1].shape)
    Tmp3[gridMapping[:,1]<=200] = 1

    colorPts = np.array([colorIMg[:,:,0].flatten('F'), colorIMg[:,:,1].flatten('F'), colorIMg[:,:,2].flatten('F')]).transpose()
    validPix = np.zeros(gridMapping[:,0].shape)

    tmp = []
    tmp2 = []
    tmp3 = [[],[],[]]

    for i in range(validPix.shape[0]):
        if Tmp0[i] == 1 and Tmp1[i] == 1 and Tmp2[i] == 1 and Tmp3[i] == 1:
            validPix[i] = 1
            tmp.append(int(gridMapping[i,1])-1)
            tmp2.append(int(gridMapping[i,0])-1)
            tmp3[0].append(colorPts[i,0])
            tmp3[1].append(colorPts[i,1])
            tmp3[2].append(colorPts[i,2])

    tmp = np.array(tmp)
    tmp2 = np.array(tmp2)
    arr = np.array([tmp, tmp2])
    s = np.ravel_multi_index(arr, (200,300), order='F')
    tmp3 = np.array(tmp3).transpose()

    for i in range(s.shape[0]):
        heightMapColor[s[i], :] = tmp3[i,:]

    return heightMapColor

def heightmapwithbgsubtraction(gridMapping, fgMask, depthImg):
    heightMap = np.zeros((200,300))
    tmp0 = np.zeros(gridMapping[:,0].shape)
    tmp0[gridMapping[:,0]>0] = 1

    tmp1 = np.zeros(gridMapping[:,0].shape)
    tmp1[gridMapping[:,0]<=300] = 1

    tmp2 = np.zeros(gridMapping[:,1].shape)
    tmp2[gridMapping[:,1]>0] = 1

    tmp3 = np.zeros(gridMapping[:,1].shape)
    tmp3[gridMapping[:,1]<=200] = 1

    tmp4 = np.zeros(gridMapping[:,2].shape)
    tmp4[gridMapping[:,2]>0] = 1

    validPix = np.zeros(gridMapping[:,0].shape)
    validPix[((tmp0 == 1) & (tmp1 == 1) & (tmp2 == 1) & (tmp3 == 1) & (tmp4 == 1))] = 1

    validDepth = np.zeros(fgMask.shape)
    validDepth[((fgMask != 0) & (depthImg != 0))] = 1

    tmp = validDepth.flatten('F')
    tmp3 = [[],[],[]]
    for i in range(validPix.shape[0]):
        if validPix[i] != 0 and tmp[i]!= 0:
            tmp3[0].append(gridMapping[i,0])
            tmp3[1].append(gridMapping[i,1])
            tmp3[2].append(gridMapping[i,2])
    gridMapping = np.array(tmp3).transpose()

    tmp = []
    tmp2 = []
    for i in range(gridMapping.shape[0]):
        tmp.append(int(gridMapping[i,1])-1)
        tmp2.append(int(gridMapping[i,0])-1)

    tmp = np.array(tmp)
    tmp2 = np.array(tmp2)
    arr = np.array([tmp, tmp2])
    s = np.ravel_multi_index(arr, (200,300), order='F')
    tmp3 = []
    for i in range(gridMapping[:,2].shape[0]):
        tmp3.append(gridMapping[:,2][i])

    for i in range(s.shape[0]):
        coord = np.unravel_index(s[i], (200,300), order='F')
        heightMap[coord[0],coord[1]] = tmp3[i]

    return heightMap

def FixMissingDepth2camera(depthImg, bgDepthImg, camIntrinsics, camPose):
    missingDepth = np.zeros(depthImg.shape)
    missingDepth[((depthImg) == 0 & (bgDepthImg > 0))] = 1

    xvalues = np.linspace(1,640,640)
    yvalues = np.linspace(1,480,480)
    pixX,pixY = np.meshgrid(xvalues, yvalues)

    camX = (pixX - camIntrinsics[0][2])*(bgDepthImg/camIntrinsics[0][0])
    camY = (pixY - camIntrinsics[1][2])*(bgDepthImg/camIntrinsics[1][1])
    camZ = bgDepthImg

    tmp = [[],[],[]]
    for i in range(missingDepth.shape[1]):
        for j in range(missingDepth.shape[0]):
            if missingDepth[j, i] == 1:
                tmp[0].append(camX[j, i])
                tmp[1].append(camY[j, i])
                tmp[2].append(camZ[j, i])

    missingCamPts = np.array(tmp).transpose()

    g = np.expand_dims(camPose[0:3,3], axis=1)
    tmp = np.repeat(g,missingCamPts.shape[0], axis=1).transpose()
    tmp2 = np.dot(camPose[0:3,0:3], missingCamPts.transpose()).transpose()
    missingCamPtsworldPts = tmp2 + tmp

    return missingDepth, missingCamPts, missingCamPtsworldPts

def getMissingdepthheightmap(binMiddleBottom, missingCamPtsworldPts):
    voxelSize = 0.002
    missingHeightMap = np.zeros((200,300))
    gridOrigin = [binMiddleBottom[0][0]-0.3,binMiddleBottom[0][1]-0.2,binMiddleBottom[0][2]]
    gridMapping_1 = np.around((missingCamPtsworldPts[:,0] - gridOrigin[0])/voxelSize)
    gridMapping_2 = np.around((missingCamPtsworldPts[:,1] - gridOrigin[1])/voxelSize)
    gridMapping_3 = (missingCamPtsworldPts[:,2] - binMiddleBottom[0][2])
    gridMapping = np.array([gridMapping_1, gridMapping_2, gridMapping_3]).transpose()

    tmp0 = np.zeros(gridMapping[:,0].shape)
    for i in range(tmp0.shape[0]):
        if gridMapping[i,0] > 0:
            tmp0[i] = 1

    tmp1 = np.zeros(gridMapping[:,0].shape)
    for i in range(tmp1.shape[0]):
        if gridMapping[i,0] <= 300:
            tmp1[i] = 1

    tmp2 = np.zeros(gridMapping[:,1].shape)
    for i in range(tmp2.shape[0]):
        if gridMapping[i,1] > 0:
            tmp2[i] = 1

    tmp3 = np.zeros(gridMapping[:,1].shape)
    for i in range(tmp3.shape[0]):
        if gridMapping[i,1] <=200:
            tmp3[i] = 1

    tmp = [[],[],[]]
    validPix = np.zeros(gridMapping[:,0].shape)
    for i in range(validPix.shape[0]):
        if tmp0[i] == 1 and tmp1[i] == 1 and tmp2[i] == 1 and tmp3[i] == 1:
            validPix[i] = 1
            tmp[0].append(gridMapping[i,0])
            tmp[1].append(gridMapping[i,1])
            tmp[2].append(gridMapping[i,2])

    gridMapping = np.array(tmp).transpose()

    tmp = []
    tmp2 = []
    for i in range(gridMapping.shape[0]):
        tmp.append(int(gridMapping[i,1])-1)
        tmp2.append(int(gridMapping[i,0])-1)

    tmp = np.array(tmp)
    tmp2 = np.array(tmp2)
    arr = np.array([tmp, tmp2])
    s = np.ravel_multi_index(arr, (200,300), order='F')
    tmp3 = []
    for i in range(gridMapping[:,2].shape[0]):
        tmp3.append(gridMapping[:,2][i])

    for i in range(s.shape[0]):
        coord = np.unravel_index(s[i], (200,300), order='F')
        missingHeightMap[coord[0],coord[1]] = 1


    missingHeightMap = missingHeightMap.astype('uint8')
    g = np.zeros(missingHeightMap.shape).astype('uint8')
    g[missingHeightMap > 0] = 1
    tmp = bwareaopen(g, 50)
    L = np.ones(missingHeightMap.shape).astype('uint8')
    noisePix = L - tmp
    missingHeightMap[noisePix==1] = 0

    return missingHeightMap, noisePix

def denoiseheightmap(heightMap, noisePix):
    heightMap = heightMap.astype('uint8')
    g = np.zeros(heightMap.shape).astype('uint8')
    g[heightMap > 0] = 1
    tmp = bwareaopen(g, 50)
    L = np.ones(heightMap.shape).astype('uint8')
    noisePix = L - tmp
    heightMap[noisePix==1] = 0

    return heightMap

Load data

In [57]:
path = '/home/arg/handover_grasping/sample'
ColorIMg = cv2.imread(path+'/color-input/000000-0.png')/255.0
ColorIMg = ColorIMg[:,:,[2,1,0]]
DepthImg = imageio.imread(path+'/depth-input/000000-0.png')/10000.0
BgColorImg = cv2.imread(path+'/color-background/000000-0.png')/255.0
BgColorImg = BgColorImg[:,:,[2,1,0]]
BgDepthImg = imageio.imread(path+'/depth-background/000000-0.png')/10000.0
CamIntrinsics = np.loadtxt(path+'/camera-intrinsics/000000-0.txt')

CamPose = np.loadtxt(path+'/camera-pose/000000-0.txt')
BinMiddleBottom = np.loadtxt(path+'/bin-position.txt')
BinMiddleBottom = np.expand_dims(BinMiddleBottom, axis=0)

In [58]:
import time

Do background subtraction

In [134]:
s = time.time()
FgMaskColor, FgMask = bg_subtraction(ColorIMg, BgColorImg, DepthImg, BgDepthImg)
print(time.time()-s)

0.020553112030029297


Project depth into camera space

In [45]:
s = time.time()
CamPts = project_depth2camera(CamIntrinsics, CamPose, DepthImg)
print(time.time()-s)

0.014075279235839844


Transform points to world coordinates

In [46]:
s = time.time()
WorldPts = points2world(CamPose, CamPts)
print(time.time()-s)

0.014129638671875


Get height map

In [47]:
s = time.time()
GridMapping = getgridmapping(WorldPts, BinMiddleBottom)
print(time.time()-s)

0.004257678985595703


Compute height map color

In [136]:
s = time.time()
HeightMapColor = getheightmapColor(GridMapping, ColorIMg)
print(time.time()-s)

0.8493330478668213


Compute real height map with background subtraction

In [137]:
s = time.time()
HeightMap = heightmapwithbgsubtraction(GridMapping, FgMask, DepthImg)
print(time.time()-s)

0.8525211811065674


Find missing depth and project background depth into camera space

In [143]:
s = time.time()
MissingDepth, MissingCamPts, MissingCamPtsworldPts = FixMissingDepth2camera(DepthImg, BgDepthImg, CamIntrinsics, CamPose)
print(time.time()-s)

0.1672658920288086


Get missing depth height map

In [138]:
s = time.time()
MissingHeightMap, NoisePix = getMissingdepthheightmap(BinMiddleBottom, MissingCamPtsworldPts)
print(time.time()-s)

0.04289674758911133


Denoise height map

In [129]:
s = time.time()
HeightMap = denoiseheightmap(HeightMap, NoisePix)
print(time.time()-s)

0.009256601333618164


Save result

In [130]:
HeightMapColor = np.reshape(HeightMapColor, (200,300,3), order='F')
colorData = np.zeros((224,320,3))
colorData[12:212,10:310,:] = HeightMapColor[:,:,[2,1,0]]
colorData = colorData*255

In [131]:
cv2.imwrite(path+'/heightmap_color.png', colorData)

True

In [17]:
HeightMap[HeightMap>0.3] = 0.3
rawHeightMap = HeightMap

In [18]:
depthData = np.zeros((224,320)).astype('uint16')
depthData[12:212,10:310] = rawHeightMap*10000

In [19]:
cv2.imwrite(path+'/heightmap_depth.png', depthData)

True